In [1]:
import numpy as np
import random
import gymnasium as gym
import Gym_Env

def epsilon_greedy_action(Q, epsilon, state):
    if np.random.rand() < 1 - epsilon:
        # p(a = a*|s) = 1 - epsilon + epsilon / |A(s)|
        action = Action[Q[state[0],state[1]].argmax()]
    else:
        # p(a = a', a'!= a*|s) = epsilon / |A(s)|
        action = Action[np.random.choice(range(4))]
    return action

In [2]:
# 定义数字对应的Action与R_Action，方便动作价值函数 在选择动作时 对应
Action = {0:"up", 1:"down", 2:"left", 3:"right"}
R_Action = {"up":0, "down":1, "left":2, "right":3}

def SARSA(env, num_episodes=500, max_episode_len=2000, gamma=0.9, epsilon=0.1, alpha=0.5):
    # 1 初始动作价值函数Q，
    # 10,7 表示状态， 4 表明种动作，7行，10列
    # Q = np.random.uniform(-1, 0, (7, 10, 4)) # zeros(shape)
    # Q[3,7] = np.array([0., 0., 0., 0.])
    
    Q = np.zeros((7,10,4))
    
    reward_lis = []
    for i in range(num_episodes):
        # 记录一下 reward
        R = 0
        # 2 初始化状态，对于 gym 环境编写种，每次reset()都会初始化agent位置为(0,3)
        state = np.array([3, 0])
        env.reset()
        # 2 选择动作 根据 Q(epsilon-greedy) 进行动作选择
        action = epsilon_greedy_action(Q, epsilon, state)

        steps = 0
        terminated = False
        # 终止条件设置为要么步数达到限制，要么agent已经抵达了目标
        while (steps <= max_episode_len) & (terminated==False):
            steps += 1

            # 3 执行动作，并且返还观察结果，奖励，以及是否终止
            observation, reward, terminated, _, _ = env.step(action)
            R += reward
            
            if terminated == True:
                print("finished")
            
            # 3 利用 Q (epsilon-greedy) 进行更新的动作选择，依据上图给出的算法
            next_state = observation["agent"]
            next_action = epsilon_greedy_action(Q, epsilon, next_state)

            # 更新当前 状态与动作 对应的值函数
            TD_error = reward + gamma * Q[next_state[0], next_state[1], R_Action[next_action]] - Q[state[0], state[1], R_Action[action]]
            Q[state[0], state[1], R_Action[action]] += alpha * TD_error

            # 更新动作与状态
            state, action = next_state, next_action
        
        reward_lis.append(R)
            
        #epilon 的探索策略，随着episodes增加，会越来越小，但最小是0.05
        epsilon = max(epsilon * 0.99995, 0.05)
    
    # 算法运行结束后, argmax(2), 表示在第二个维度上最大化，即动作维度上
    policy = Q.argmax(2)
    
    return Q, policy, reward_lis

In [ ]:
# 个人目前不太能理解，为何对于状态空间转换为矩阵时，会出现错误
# 将每一个状态转换为标签，最终能够正确执行代码